In [1]:
#CSC 4402 PROGRAMMING ASSIGNMENT 1
#ANNA WILLIAMS awil459@lsu.edu

import heapq
from enum import Enum

In [2]:
class ACTION(Enum):
    UP=1
    DOWN=2
    LEFT=3
    RIGHT=4
    SUCK=5

class STATE:
    def __init__(self, grid,position,parent,action_taken):
        self.grid=grid
        self.position=position
        self.parent=parent
        self.action_taken=action_taken
        self.g=0
        self.h=0
        self.f=0
    def __lt__(self,other):          #Less than comparator function needed for adding states to priority queue based on their f value
        return self.f<other.f
    def __eq__(self,other):
        return self.grid==other.grid and self.position==other.position #Equality comparator function needed to ensure successor state not already in expanded state list

In [3]:
# FORMULATING THE COST AND HEURISTIC FUNCTIONS
# in order to calculate the cost of moving from a state s to a state t, we need to calculate 1 thing:
# [1] the number of dirty squares at state s
# in order to calculate the heuristic value of a state s, we need to calculate two things:
# [1] the distance to the nearest dirty square from the position of the vacuum in state s
# [2] the number of dirty squares at state s

# function get_dirty_squares:
# input: grid: a binary list where:
#       grid[i]=0  -> the square at position i is clean
#       grid[i]=1  -> the square at position i is dirty
# output: dirty_squares: a list of size m, where:
#        m=the number of dirty squares in grid
#        dirty_squares[i]=j  -> the square located at position j in list grid is dirty
def get_dirty_squares(grid):
    dirty_squares=[]
    length=len(grid)
    for i in range(length):
        if grid[i]==1:
            dirty_squares.append(i)
    return dirty_squares

# function get_manhattan_distance:
# NOTE: because we are using a 1-dimensional list to represent a 2D space, we need a special method to calculate
# the manhattan distance between squares.
# we are assuming our vacuum world is an nxn square matrix and representing it as a list of length n^2
# the matrix coordinates of a given index in the list can be calculated as follows:
#   let k=the index of any element in our list of length n^2
#   then k/n=i is the row index of our square in the matrix space
#   and k%n=j is the column index of our square in the matrix space
# input: position1, position2: integers whose values represent list indecies
# output: the manhattan distance between the two positions
def get_manhattan_distance(position1,position2):
    position1_i=position1//5
    position1_j=position1%5
    position2_i=position2//5
    position2_j=position2%5
    return abs(position1_i-position2_i)+abs(position1_j-position2_j)

# function get_nearest_dirty
# input: position: integer value representing the current position of the vacuum agent
#        dirty_squares: list which contains the indecies of the dirty squares
# output: min_distance: the manhattan distance between the vacuum agent and the nearest dirty square
def get_nearest_dirty(position,dirty_squares):
    if not dirty_squares:
        return 0
    min_distance=min(
        get_manhattan_distance(position,dirt)
        for dirt in dirty_squares
    )
    return min_distance

In [4]:
# FORMULATING THE SUCCESSOR FUNCTION

# function successor_fn:
# input: current_state: an object of class STATE, representing the agents current position and the current grid of clean and dirty squares
#        action:        an object of class ACTION representing one of 5 possible actions: UP, DOWN, LEFT, RIGHT, SUCK
#        if UP is chosen, the agent's position is decremented by 5 if it is not already in the top row
#        if DOWN is chosen, the agents position is incremented by 5 if it is not already in the bottom row
#        if LEFT is chosen, the agents position is decremented by 1 if it is not already in the leftmost column
#        if RIGHT is chosen, the agents position is incremented by 1 if it is not already in the rightmost column
#        if SUCK is chosen, the agent cleans the square represented by its current position
# output: new_state: an object of class STATE, representing the new state the agent enters based on the action it takes
# the g value (cost) of the new state is calculated in this function as:
# the g value of the current sate + 2*the number of dirty squares remaining + the step cost, 1
def successor_fn(current_state,action):
    new_position=current_state.position
    new_grid=[]
    for i in range(len(current_state.grid)):
        new_grid.append(current_state.grid[i])
    if action==ACTION.UP:
        if current_state.position>4:
            new_position=current_state.position-5
    elif action==ACTION.DOWN:
        if current_state.position<20:
            new_position=current_state.position+5
    elif action==ACTION.LEFT:
        if current_state.position%5 !=0:
            new_position=current_state.position-1
    elif action==ACTION.RIGHT:
        if (current_state.position+1)%5 !=0:
            new_position=current_state.position+1
    elif action==ACTION.SUCK:
        if current_state.grid[current_state.position]==1:
            new_grid[current_state.position]=0
    new_state=STATE(new_grid,new_position,parent=current_state,action_taken=action)
    dirty_squares=get_dirty_squares(new_state.grid)
    new_state.g=current_state.g+(2*len(dirty_squares))+1
    return new_state

In [10]:
# PART A HEURISTIC FUNCTION 
def heuristic1(current_state):
    dirty_squares=get_dirty_squares(current_state.grid)
    distance=get_nearest_dirty(current_state.position,dirty_squares)
    dirt_penalty=2*len(dirty_squares)
    return distance+dirt_penalty-1

# PART B HEURISTIC FUNCTION
def heuristic2(current_state):
    dirty_squares=get_dirty_squares(current_state.grid)
    num_dirty_squares=len(dirty_squares)
    distance=get_nearest_dirty(current_state.position,dirty_squares)
    return (distance*2*num_dirty_squares)+(2*num_dirty_squares*(num_dirty_squares-1))

In [25]:
def print_optimal_path(path):
    length=len(path)
    print('[*] CURRENTLY IN INITIAL STATE')
    print('[*] AGENTS CURRENT POSITION IS: ',(path[0].position))
    print('[*] CURRENT STATUS OF THE ENVIRONMENT:')
    print(path[0].grid)
    print('[*] F VALUE OF CURRENT STATE IS: ',(path[0].f))
    print('[*] ACTION CHOSEN BY A* SEARCH: ' ,(path[1].action_taken.name))
    print('----------------------------------------------------------------------')
    for i in range(1,length-1):
        print('[*] CURRENTLY IN STATE: ' ,(i))
        print('[*] AGENTS CURRENT POSITION IS: ',(path[i].position))
        print('[*] CURRENT STATUS OF THE ENVIRONMENT:')
        print(path[i].grid)
        print('[*] F VALUE OF CURRENT STATE IS: ',(path[i].f))
        print('[*] ACTION CHOSEN BY A* SEARCH: ' ,(path[i+1].action_taken.name))
        print('----------------------------------------------------------------------')
    print('[*] CURRENTLY IN FINAL STATE')
    print('[*] AGENTS CURRENT POSITION IS: ',(path[length-1].position))
    print('[*] CURRENT STATUS OF THE ENVIRONMENT:')
    print(path[length-1].grid)
    print('[*] F VALUE OF CURRENT STATE IS: ',(path[length-1].f))
    print('----------------------------------------------------------------------')


In [31]:
# PART A: A* FUNCTION
# input: init_state, an object of class STATE
# output: none
# the function adds the initial state to a priority queue of unexpanded states
# while there are still unexpanded states, the function checks the unexpanded state with the lowest f value to see if it is the goal state
# if the current state is the goal state, it prints the optimal path to this state
# otherwise, the function searches the successors of the current state
# if a successor state has not already been previously expanded, it is added to a list of successors to potentially be expanded
# each successor in this list of this successor then has its f value calculated
# if a successor state is not already in the queue of unexpanded states, or if it is in the queue with a higher f value, then the successor state is added to the queue
# the current state is added to the list of expanded states
# the successor state with the lowest f value becomes the current state, and the function repeats until the goal state is reached 
def a_star_partA(init_state):
    goal_grid=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    expanded_states=[]
    unexpanded_states=[]
    optimal_path=[]
    init_state.h=heuristic1(init_state)
    init_state.f=init_state.g+init_state.h
    unexpanded_states.append(init_state)
    while unexpanded_states:
        current_state=heapq.heappop(unexpanded_states)
        if current_state.grid==goal_grid:
            while current_state is not None:
                optimal_path.append(current_state)
                current_state=current_state.parent             
            print_optimal_path(optimal_path[::-1])
            return
        successors=[]
        for action in (ACTION):
            successor=successor_fn(current_state,action)
            if successor not in expanded_states:
                successors.append(successor)
        for successor in successors:
            successor.h=heuristic1(successor)
            successor.f=successor.g+successor.h
            if successor not in unexpanded_states:
                heapq.heappush(unexpanded_states,successor)
            else:
                for unexpanded_state in unexpanded_states:
                    if successor.f<unexpanded_state.f:
                        heapq.heappush(unexpanded_states,successor)
                    else:
                        break
        expanded_states.append(current_state)

In [32]:
# PART B: A* FUNCTION
# this function is the same as the function in part A, except it calculates the heuristic value of each state using the heuristic2 function
def a_star_partB(init_state):
    goal_grid=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    expanded_states=[]
    unexpanded_states=[]
    optimal_path=[]
    init_state.h=heuristic2(init_state)
    init_state.f=init_state.g+init_state.h
    unexpanded_states.append(init_state)
    while unexpanded_states:
        current_state=heapq.heappop(unexpanded_states)
        if current_state.grid==goal_grid:
            while current_state is not None:
                optimal_path.append(current_state)
                current_state=current_state.parent             
            print_optimal_path(optimal_path[::-1])
            return
        successors=[]
        for action in (ACTION):
            successor=successor_fn(current_state,action)
            if successor not in expanded_states:
                successors.append(successor)
        for successor in successors:
            successor.h=heuristic2(successor)
            successor.f=successor.g+successor.h
            if successor not in unexpanded_states:
                heapq.heappush(unexpanded_states,successor)
            else:
                for unexpanded_state in unexpanded_states:
                    if successor.f<unexpanded_state.f:
                        heapq.heappush(unexpanded_states,successor)
                    else:
                        break
        expanded_states.append(current_state)

In [33]:
# PART A: running A* search using heuristic function 1
init_grid=[1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
init_position=20
init_state=STATE(init_grid,init_position,parent=None,action_taken=None)
a_star_partA(init_state)

[*] CURRENTLY IN INITIAL STATE
[*] AGENTS CURRENT POSITION IS:  20
[*] CURRENT STATUS OF THE ENVIRONMENT:
[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[*] F VALUE OF CURRENT STATE IS:  13
[*] ACTION CHOSEN BY A* SEARCH:  UP
----------------------------------------------------------------------
[*] CURRENTLY IN STATE:  1
[*] AGENTS CURRENT POSITION IS:  15
[*] CURRENT STATUS OF THE ENVIRONMENT:
[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[*] F VALUE OF CURRENT STATE IS:  23
[*] ACTION CHOSEN BY A* SEARCH:  UP
----------------------------------------------------------------------
[*] CURRENTLY IN STATE:  2
[*] AGENTS CURRENT POSITION IS:  10
[*] CURRENT STATUS OF THE ENVIRONMENT:
[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[*] F VALUE OF CURRENT STATE IS:  33
[*] ACTION CHOSEN BY A* SEARCH:  UP
----------------------------------------------------------------------
[*] CURRENTLY IN STATE:  3
[*]

In [34]:
# PART B: running A* search using heuristic function 2
init_grid=[1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
init_position=20
init_state=STATE(init_grid,init_position,parent=None,action_taken=None)
a_star_partB(init_state)

[*] CURRENTLY IN INITIAL STATE
[*] AGENTS CURRENT POSITION IS:  20
[*] CURRENT STATUS OF THE ENVIRONMENT:
[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[*] F VALUE OF CURRENT STATE IS:  80
[*] ACTION CHOSEN BY A* SEARCH:  UP
----------------------------------------------------------------------
[*] CURRENTLY IN STATE:  1
[*] AGENTS CURRENT POSITION IS:  15
[*] CURRENT STATUS OF THE ENVIRONMENT:
[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[*] F VALUE OF CURRENT STATE IS:  81
[*] ACTION CHOSEN BY A* SEARCH:  UP
----------------------------------------------------------------------
[*] CURRENTLY IN STATE:  2
[*] AGENTS CURRENT POSITION IS:  10
[*] CURRENT STATUS OF THE ENVIRONMENT:
[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[*] F VALUE OF CURRENT STATE IS:  82
[*] ACTION CHOSEN BY A* SEARCH:  UP
----------------------------------------------------------------------
[*] CURRENTLY IN STATE:  3
[*]